In [19]:
import json

json_file_path = '/app/Law_corpus/law-corpus.json'

with open(json_file_path, 'rt') as f_in:
    docs_raw = json.load(f_in)
# docs_raw = [docs_raw]
docs_raw

documents = []

for law_title, law_data in docs_raw.items():
    law_title = law_data.get("law_title")
    law_number = law_data.get("law_number")
    chapters = law_data.get("chapters", [])
    
    for chapter in chapters:
        chapter_title = chapter.get("chapter_title")
        articles = chapter.get("articles", [])
        
        for article in articles:
            doc = {
                "law_title": law_title,
                "law_number": law_number,
                "chapter_title": chapter_title,
                "article_number": article.get("article_number"),
                "title": article.get("title"),
                "content": article.get("content")
            }
            documents.append(doc)

In [7]:
documents[0]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 1',
 'title': 'Phạm vi điều chỉnh',
 'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. '}

In [8]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['law_title']}-{doc['chapter_title']}-{doc['title']}-{doc['content'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [9]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [10]:
documents[3]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 4',
 'title': 'Phát triển giáo dục',
 'content': '1. Phát triển giáo dục là quốc sách hàng đầu. 2. Phát triển giáo dục phải gắn với nhu cầu phát triển kinh tế - xã hội, tiến bộ khoa học, công nghệ, củng cố quốc phòng, an ninh; thực hiện chuẩn hóa, hiện đại hóa, xã hội hóa; bảo đảm cân đối cơ cấu ngành nghề, trình độ, nguồn nhân lực và phù hợp vùng miền; mở rộng quy mô trên cơ sở bảo đảm chất lượng và hiệu quả; kết hợp giữa đào tạo và sử dụng. 3. Phát triển hệ thống giáo dục mở, xây dựng xã hội học tập nhằm tạo cơ hội để mọi người được tiếp cận giáo dục, được học tập ở mọi trình độ, mọi hình thức, học tập suốt đời. ',
 'id': '06ce8ae6'}

In [11]:
from collections import defaultdict

In [12]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [13]:
len(hashes), len(documents)

(314, 314)

In [14]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [15]:
hashes['593f7569']

[]

In [16]:
import json
import os

In [17]:
output_dir = "./Law_corpus"
output_path = os.path.join(output_dir, 'documents-with-ids.json')

with open(output_path, 'wt') as f_out:
    json.dump(documents, f_out, ensure_ascii=False, indent=4)

In [12]:
!head ./Law_corpus/documents-with-ids.json

[
    {
        "law_title": "Luật Giáo Dục",
        "law_number": "Luật số: 43/2019/QH14",
        "chapter_title": "Chương I",
        "article_number": "Điều 1",
        "title": "Phạm vi điều chỉnh",
        "content": "Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ",
        "id": "75a9286e"
    },


In [13]:
import json

with open('/app/Law_corpus/documents-with-ids.json', 'rt') as f_in:  # Use /app, not the Windows path
    documents = json.load(f_in)

In [15]:
documents[0]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 1',
 'title': 'Phạm vi điều chỉnh',
 'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ',
 'id': '75a9286e'}

In [24]:
prompt_template = """
Bạn mô phỏng một người dùng đang tìm kiếm trợ lý luật.
Dựa trên hồ sơ Luật Việt Nam được cung cấp, hãy tạo ra **1 cặp câu hỏi và câu trả lời** (question-answer pair).
Câu hỏi phải liên quan trực tiếp đến nội dung của hồ sơ luật, và câu trả lời phải được trích xuất từ thông tin trong hồ sơ.
Hãy đảm bảo rằng câu hỏi đầy đủ và rõ ràng, và câu trả lời vừa đủ, không dài không ngắn, chính xác.

Hồ sơ luật:

law_title: {law_title}
title: {title}
content: {content}

Cung cấp đầu ra dưới dạng JSON có thể phân tích được mà không sử dụng các khối mã:

{{
  "question": "Câu hỏi được tạo dựa trên hồ sơ luật",
  "answer": "Câu trả lời được trích xuất từ hồ sơ luật"
}}
""".strip()

In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() 
DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")
client = OpenAI(
    api_key = DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com"
)

In [21]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=False
    )

    json_response = response.choices[0].message.content
    return json_response

In [25]:
from tqdm.auto import tqdm

# Initialize an empty dictionary to store results
results = {}

# Select only the first document for testing
test_document = [documents[-1]]

# Loop through the test document (only 1 document in this case)
for doc in tqdm(test_document): 
    doc_id = doc['id']
    
    # Skip if the document ID is already in results (not necessary here, but kept for consistency)
    if doc_id in results:
        continue

    # Generate questions for the document
    questions = generate_questions(doc)
    
    # Store the result
    results[doc_id] = questions

# Print the results for verification
print(results)

  0%|          | 0/1 [00:00<?, ?it/s]

{'2bb7dafa': '```json\n{\n  "question": "Ai là người ký thông qua Luật Giao Thông Đường Bộ được Quốc hội khóa XII, kỳ họp thứ 4 thông qua ngày 13 tháng 11 năm 2008?",\n  "answer": "CHỦ TỊCH QUỐC HỘI Nguyễn Phú Trọng"\n}\n```'}


In [17]:
from tqdm.auto import tqdm

In [18]:
results = {}

In [25]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/314 [00:00<?, ?it/s]

In [43]:
results

{'75a9286e': '[\n    "Luật Giáo Dục điều chỉnh những đối tượng nào trong hệ thống giáo dục quốc dân?",\n    "Những quyền và trách nhiệm nào của cá nhân liên quan đến hoạt động giáo dục được quy định trong Luật này?",\n    "Cơ sở giáo dục và nhà giáo thuộc phạm vi điều chỉnh của Luật Giáo Dục không?",\n    "Luật quy định về vấn đề quản lý nhà nước trong giáo dục như thế nào?",\n    "Những hoạt động giáo dục nào được Luật Giáo Dục quy định cụ thể?"\n]',
 '82b52978': '[\n  "Mục tiêu giáo dục của Việt Nam là gì?",\n  "Luật Giáo Dục xác định những phẩm chất nào cần có ở học sinh?",\n  "Làm thế nào để phát triển tiềm năng và khả năng sáng tạo của học sinh?",\n  "Luật Giáo Dục có nhấn mạnh về lòng yêu nước hay không?",\n  "Mục tiêu giáo dục có liên quan gì đến sự nghiệp xây dựng Tổ quốc và hội nhập quốc tế?"\n]',
 'e0e71ffa': '[\n  "Nền giáo dục Việt Nam có đặc điểm gì nổi bật?",\n  "Nguyên lý giáo dục nào được áp dụng trong hệ thống giáo dục Việt Nam?",\n  "Ai là nền tảng tư tưởng của nền gi

In [ ]:
doc_id = '35bdafa7'
questions = generate_questions(doc)
results[doc_id] = questions

In [33]:
import pickle
with open('results.bin', 'wb') as f_out:
    pickle.dump(results, f_out)


In [38]:
with open('./results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [45]:
results['35bdafa7']

'["Trường của cơ quan nhà nước có nhiệm vụ gì trong việc đào tạo cán bộ?", "Ai là đối tượng được đào tạo tại trường lực lượng vũ trang nhân dân?", "Luật nào quy định hoạt động của các trường trong hệ'

In [ ]:
json_questions = [
  r"Trường của cơ quan nhà nước, tổ chức chính trị, tổ chức chính trị - xã hội có nhiệm vụ chính là gì?",
  "Trường của lực lượng vũ trang nhân dân đào tạo những đối tượng nào?",
  "Các trường của cơ quan nhà nước, tổ chức chính trị - xã hội, và lực lượng vũ trang nhân dân hoạt động theo những quy định nào?",
  "Khi nào thì trường của các cơ quan, tổ chức này được cấp văn bằng, chứng chỉ của hệ thống giáo dục quốc dân?",
  "Cơ quan nào chịu trách nhiệm quy định chi tiết về hoạt động của các loại trường này?"
]
results['35bdafa7'] = json.dumps(json_questions)

In [50]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [54]:
parsed_resulst['75a9286e']

['Luật Giáo Dục điều chỉnh những đối tượng nào trong hệ thống giáo dục quốc dân?',
 'Những quyền và trách nhiệm nào của cá nhân liên quan đến hoạt động giáo dục được quy định trong Luật này?',
 'Cơ sở giáo dục và nhà giáo thuộc phạm vi điều chỉnh của Luật Giáo Dục không?',
 'Luật quy định về vấn đề quản lý nhà nước trong giáo dục như thế nào?',
 'Những hoạt động giáo dục nào được Luật Giáo Dục quy định cụ thể?']

In [52]:
doc_index = {d['id']: d for d in documents}

In [55]:
doc_index['75a9286e']

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 1',
 'title': 'Phạm vi điều chỉnh',
 'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ',
 'id': '75a9286e'}

In [61]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    law_title = doc_index[doc_id]['law_title']
    for q in questions:
        final_results.append((q, law_title, doc_id))

In [63]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['questions', 'law_title', 'document'])
# law_title: {law_title}
# title: {title}
# answer: {content}
df.head(5)

,questions,law_title,document
0,Luật Giáo Dục điều chỉnh những đối tượng nào t...,Luật Giáo Dục,75a9286e
1,Những quyền và trách nhiệm nào của cá nhân liê...,Luật Giáo Dục,75a9286e
2,Cơ sở giáo dục và nhà giáo thuộc phạm vi điều ...,Luật Giáo Dục,75a9286e
3,Luật quy định về vấn đề quản lý nhà nước trong...,Luật Giáo Dục,75a9286e
4,Những hoạt động giáo dục nào được Luật Giáo Dụ...,Luật Giáo Dục,75a9286e
5,Mục tiêu giáo dục của Việt Nam là gì?,Luật Giáo Dục,82b52978
6,Luật Giáo Dục xác định những phẩm chất nào cần...,Luật Giáo Dục,82b52978
7,Làm thế nào để phát triển tiềm năng và khả năn...,Luật Giáo Dục,82b52978
8,Luật Giáo Dục có nhấn mạnh về lòng yêu nước ha...,Luật Giáo Dục,82b52978
9,Mục tiêu giáo dục có liên quan gì đến sự nghiệ...,Luật Giáo Dục,82b52978


In [70]:
df.tail(5)

,questions,law_title,document
1565,Quy định chi tiết và hướng dẫn thi hành Luật G...,Luật Giao Thông Đường Bộ,2bb7dafa
1566,Luật Giao Thông Đường Bộ được Quốc hội thông q...,Luật Giao Thông Đường Bộ,2bb7dafa
1567,Ai là Người đứng đầu khi thông qua Luật Giao T...,Luật Giao Thông Đường Bộ,2bb7dafa
1568,Luật này nhằm mục đích gì trong quản lý nhà nước?,Luật Giao Thông Đường Bộ,2bb7dafa
1569,Luật có liên quan đến những nội dung nào khác ...,Luật Giao Thông Đường Bộ,2bb7dafa


In [71]:
df.to_csv('ground-truth-data.csv', index=False)

In [72]:
!head ground-truth-data.csv

questions,law_title,document
Luật Giáo Dục điều chỉnh những đối tượng nào trong hệ thống giáo dục quốc dân?,Luật Giáo Dục,75a9286e
Những quyền và trách nhiệm nào của cá nhân liên quan đến hoạt động giáo dục được quy định trong Luật này?,Luật Giáo Dục,75a9286e
Cơ sở giáo dục và nhà giáo thuộc phạm vi điều chỉnh của Luật Giáo Dục không?,Luật Giáo Dục,75a9286e
Luật quy định về vấn đề quản lý nhà nước trong giáo dục như thế nào?,Luật Giáo Dục,75a9286e
Những hoạt động giáo dục nào được Luật Giáo Dục quy định cụ thể?,Luật Giáo Dục,75a9286e
Mục tiêu giáo dục của Việt Nam là gì?,Luật Giáo Dục,82b52978
Luật Giáo Dục xác định những phẩm chất nào cần có ở học sinh?,Luật Giáo Dục,82b52978
Làm thế nào để phát triển tiềm năng và khả năng sáng tạo của học sinh?,Luật Giáo Dục,82b52978
Luật Giáo Dục có nhấn mạnh về lòng yêu nước hay không?,Luật Giáo Dục,82b52978
